<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/refactoring-sprint2/colabs/Heuristic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2024Project.git
%cd BigData2024Project
!git checkout refactoring-sprint2
%cd ..

Cloning into 'BigData2024Project'...
remote: Enumerating objects: 1348, done.
remote: Counting objects: 100% (694/694), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 1348 (delta 454), reused 480 (delta 314), pack-reused 654
Receiving objects: 100% (1348/1348), 54.39 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (717/717), done.
/content/BigData2024Project
Branch 'refactoring-sprint2' set up to track remote branch 'refactoring-sprint2' from 'origin'.
Switched to a new branch 'refactoring-sprint2'
/content


In [3]:
%cd BigData2024Project
!git pull
%cd ..

/content/BigData2024Project
Already up to date.
/content


In [4]:
%cd BigData2024Project
!git status
%cd ..

/content/BigData2024Project
On branch refactoring-sprint2
Your branch is up to date with 'origin/refactoring-sprint2'.

nothing to commit, working tree clean
/content


In [5]:
!chmod 755 /content/BigData2024Project/src/setup.sh
!/content/BigData2024Project/src/setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 36.4 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninstall

In [6]:
import sys
sys.path.append('/content/BigData2024Project/src')

In [7]:
from start_spark import initialize_spark
initialize_spark()

In [16]:
from pyspark.sql import SparkSession
from big_mess.heuristic_classifier import heuristic_classify
from big_mess.loaders import preprocessed_loader, save_to_csv

In [9]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
data=preprocessed_loader(spark,'/content/drive/MyDrive/BigMess/NASA/NASA_month_preprocessed.csv')

In [11]:
data.show(5)

+---------+-------+------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+
|      lon|    lat| Rainf|    Evap|AvgSurfT|  Albedo|SoilT_10_40cm|     GVEG| PotEvap|RootMoist|SoilM_100_200cm|SoilT_40_100cm|Year|Month|
+---------+-------+------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+
|-112.0625|25.0625|1.1386|11.99051|288.2954|41.47715|     289.6995|0.1971279|137.6446| 398.8725|       208.2797|          NULL|2023|    1|
|-111.9375|25.0625| 1.159|11.40101|287.4547|41.50941|     289.5214| 0.198604|152.8735| 375.6453|       192.4112|          NULL|2023|    1|
|-111.8125|25.0625| 1.179|11.10241|287.6041|41.50537|      289.974|0.1711854|109.3242| 183.8275|       211.5441|          NULL|2023|    1|
|-111.6875|25.0625|1.2057|11.20511|287.3168|41.50134|     289.8952|0.1711854|113.2493| 182.3848|       209.8121|          NULL|2023|    1|
|-111.5625|25.0625|1.3994|5

In [14]:
result = heuristic_classify(data)
result.show(20)

+---------+-------+---------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+---------------+--------------+--------------+------------------+----------------+------------------------+--------+
|      lon|    lat|    Rainf|    Evap|AvgSurfT|  Albedo|SoilT_10_40cm|     GVEG| PotEvap|RootMoist|SoilM_100_200cm|SoilT_40_100cm|Year|Month|Rainf_condition|Evap_condition|GVEG_condition|AvgSurfT_condition|Albedo_condition|conditions_fullfiled_sum|Pustynia|
+---------+-------+---------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+---------------+--------------+--------------+------------------+----------------+------------------------+--------+
|-112.0625|25.0625|   1.1386|11.99051|288.2954|41.47715|     289.6995|0.1971279|137.6446| 398.8725|       208.2797|          NULL|2023|    1|              1|             1|             1|                 0|               1|   

In [17]:
save_to_csv(result.drop("Rainf_condition","Evap_condition","GVEG_condition","AvgSurfT_condition","Albedo_condition","conditions_fullfiled_sum"), '/content/drive/MyDrive/BigMess/NASA/NASA_heuristic.csv')